# [LSTM 실습: 다변량 시계열 예측 모델링 및 비교]

### jupyter notebook 단축키

- ctrl+enter: 셀 실행   
- shift+enter: 셀 실행 및 다음 셀 이동   
- alt+enter: 셀 실행, 다음 셀 이동, 새로운 셀 생성
- a: 상단에 새로운 셀 만들기
- b: 하단에 새로운 셀 만들기
- dd: 셀 삭제(x: 셀 삭제)
- y: Code로 변경
- m: Markdown으로 변경

### 1. 모듈 불러오기

In [ ]:
from google.colab import drive
drive.mount('content/gdrive/')
import os
os.chdir('/content/gdrove/My Drive/Day3/hands-on/3일차_RNN2/')

In [ ]:
import pandas as pd
import pandas_datareader.data as pdr
# pip install pandas-datareader

import datetime
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np
import argparse
from copy import deepcopy # Add Deepcopy for args
from sklearn.metrics import mean_absolute_error

import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

print(torch.__version__)
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 9)

#### 정규화 함수

In [ ]:
# scaling function for input data
def minmax_scaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    return numerator / (denominator + 1e-7)

### 2. 데이터 불러오기
##### Pandas Datareader 사용: 야후에서 제공하는 API사용
#####  
#####  

- X: 주식 정보(High, Low, Open, Close, Volumne, Adj Close)
- y: 주식 정보(High, Low, Open, Close, Volumne, Adj Close) 중 택 1

In [ ]:
# We will look at stock prices over the past year
start = (2000, 12, 1)
start = datetime.datetime(*start) #그냥 tuple로 넣어주면 안됨, *: 인자를 각각 순서대로 넣어줌
end = datetime.date.today()

# google = pdr.DataReader('어떤종목', '어디서', 언제부터, 언제까지)                    
yahoo = pdr.DataReader('AAPL', 'yahoo', start, end)

# 한화: 000880.KS
# 한화 케미칼: 009830.KS
# 한화 손해보험: 000370.KS
# 모나미: 005360.KS
# 하이트진로홀딩스우: 000145.KS

#### 데이터 탐색

In [ ]:
print(yahoo.head())
# High: 장 중 제일 높았던 주가(고가)
# Low: 장 중 제일 낮았던 주가(저가)
# Open: 장 시작 때 주가(시가)
# Close: 장 닫을 때 주가(종가)
# Volume: 주식 거래량
# Adj Close: 주식의 분할, 배당, 배분 등을 고려해 조정한 종가

In [ ]:
print(yahoo.describe())

In [ ]:
yahoo.Close.plot(grid=True)  # Close --> change

In [ ]:
data_rvs = yahoo

#### hyperparameters
- seq_length: 시퀀스 길이
- data_dim: 변수 개수
- hidden_dim: hidden state vector 차원(=특징을 얼마나 추출하여 학습할 것인지)
- output_dim: 학습 반복 회수
- learning_rate: 학습률
- iterations: 학습 반복 회수

In [ ]:
seq_length = 7    # 반영하고자 하는 날짜
data_dim = 6      # 변수 갯수 
hidden_dim = 10
output_dim = 1    # 예측 변수 갯수
learning_rate = 0.01
iterations = 500 

In [ ]:
# split train-test set
train_size = int(len(data_rvs) * 0.7)
train_set = data_rvs[0:train_size]
test_set = data_rvs[train_size - seq_length:]

In [ ]:
# scaling data
train_set = minmax_scaler(train_set)
test_set = minmax_scaler(test_set)

In [ ]:
print(train_set.head())

### 3. 데이터 전처리: sequence 길이에 맞게  RNN Input 데이터 만들기

In [ ]:
def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, len(time_series) - seq_length):
        _x = time_series.iloc[i:i + seq_length, :]
        _y = time_series.iloc[i + seq_length, [3]]  # Next close price  #[3,4]: Close and volume
        print(_x, "->", _y)
        dataX.append(_x.values)
        dataY.append(_y.values)
    
    return np.stack(dataX), np.stack(dataY)

In [ ]:
# Remind

#seq_length = 7    # 반영하고자 하는 날짜
#data_dim = 6      # 변수 갯수 
#hidden_dim = 10
#output_dim = 1    # 예측 변수 갯수
#learning_rate = 0.01
#iterations = 500 

In [ ]:
# make train-test dataset to input
trainX, trainY = build_dataset(train_set, seq_length)
testX, testY = build_dataset(test_set, seq_length)

In [ ]:
print(trainX.shape) #3277,7,6
print(trainY.shape) #3277,1

In [ ]:
print(testX.shape) #1408,7,6
print(testY.shape) #1408,1

In [ ]:
# convert to tensor
trainX_tensor = torch.FloatTensor(trainX)
trainY_tensor = torch.FloatTensor(trainY)

testX_tensor = torch.FloatTensor(testX)
testY_tensor = torch.FloatTensor(testY)

### 4. LSTM & GRU 학습 및 평가: 다음 시점의 Close price 예측

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layers):
        super(Net, self).__init__()
        #self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.rnn = torch.nn.LSTM(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        #self.rnn = torch.nn.GRU(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        
        self.fc = torch.nn.Linear(hidden_dim, output_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x[:, -1])
        return x


net = Net(data_dim, hidden_dim, output_dim, 1)

In [ ]:
# loss & optimizer setting
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [ ]:
# start training
for i in range(iterations):

    optimizer.zero_grad()
    outputs = net(trainX_tensor)
    loss = criterion(outputs, trainY_tensor)
    loss.backward()
    optimizer.step()
    print(i, loss.item())

In [ ]:
# plot training
plt.plot(trainY)
plt.plot(net(trainX_tensor).data.numpy())
plt.legend(['original', 'prediction'])
plt.show()

In [ ]:
# start testing
for i in range(iterations):

    optimizer.zero_grad()
    test_outputs = net(testX_tensor)
    loss = criterion(test_outputs, testY_tensor)
    loss.backward()
    optimizer.step()
    print(i, loss.item())

In [ ]:
# plot testing
plt.plot(testY)
plt.plot(net(testX_tensor).data.numpy())
plt.legend(['original', 'prediction'])
plt.show()

### 5. 성능평가

In [ ]:
# solve prediction y(tensor) to numpy array
train_predictionY = outputs.detach()
train_predictionY = train_predictionY.numpy()

test_predictionY = test_outputs.detach()
test_predictionY = test_predictionY.numpy()

In [ ]:
mean_squared_error(trainY, train_predictionY)

In [ ]:
mean_squared_error(testY, test_predictionY)

In [ ]:
## RNN
## Test MSE: 6.535761378376095e-05
## Train MSE: 0.00022348110553613228


## LSTM
## Test MSE: 6.259012494261207e-05
## Train MSE: 0.00019432165494639008


## GRU
## Test MSE: 5.0685171742773086e-05
## Train MSE: 0.00023175848398944936

#### 과대적합이 발생한 상황이다.
#### 1. 은닉층 차원 줄이기
#### 2. 시퀀스 길이 늘리기
#### 3. 더 작은 학습률 적용시키기